In [6]:
!pip install pymorphy2

In [7]:
import pymorphy2
import nltk
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

morph = pymorphy2.MorphAnalyzer()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
def collect_text(url):
  html = urlopen(url).read()
  soup = BeautifulSoup(html, features="html.parser")

  # kill all script and style elements
  for script in soup(["script", "style"]):
    script.extract()    # rip it out

  # get text
  text = soup.get_text()

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  def is_date(o):
    x = o.split()
    if len(x) < 2:
      return False
    x = x[-2]
    if(len(x) == 4 and x[0] == '1' and x[1] == '9'):
      return True
    return False

  source = ''

  kill = False

  def TCHEKHOV(chunk):
    if 'ЧЕХОВ' in chunk:
      return True
    return False
  
  tch = ''

  for chunk in (chunks):
    if TCHEKHOV(chunk):
      tch = chunk
    if kill and chunk:
      source += chunk
      chunk = None
      break
    if is_date(chunk):
      source += (chunk)
      source += ' '
      chunk = None
      kill = True
    else:
      chunk = None


  erase = False

  def eraser(chunk):
    nonlocal erase
    if chunk == 'ПОИСК:':
      erase = True
    if erase:
      return False
    if not chunk:
      return False
    return True

  text = '\n'.join(chunk for chunk in chunks if eraser(chunk))

  sentences = nltk.tokenize.sent_tokenize(text)
  source = "Письмо " + tch + ' ' + source
  return source, sentences

In [9]:
def annotate_text(source, sentences):
  end_s = []
  for s in sentences:
    original = s
    original = original.lower()
    p = ',:@.-()/!'
    original_cleaned = original
    for i in p:
      original_cleaned = original_cleaned.replace(i, '')
    original_splitted = [ i for i in re.split(r'\W+',original) if i]
    
    lemmatized = []
    for j in original_splitted:
        lemmatized.append(morph.parse(j)[0].normal_form)
    tags = []
    for word in original_splitted:   
        tag = morph.parse(word)[0].tag
        tags.append(str(tag))
    end_s.append((source, s, original_splitted, lemmatized, tags))
  return end_s

In [10]:
corpora = []
for i in range(24, 354):
  if i == 193 or i == 194 or i == 202 or i == 203 or i == 204 or i == 205 or i == 313 or i == 326:
    continue
  if i < 100:
    url = 'http://apchekhov.ru/books/item/f00/s00/z0000020/st0' + str(i) + '.shtml'
  else:
    url = 'http://apchekhov.ru/books/item/f00/s00/z0000020/st' + str(i) + '.shtml'
  source, sentences = collect_text(url)
  annotated_text = annotate_text(source, sentences)
  corpora += (annotated_text)

In [ ]:
import json

data = corpora

with open('corpORAORA.txt', 'w') as outfile:
    json.dump(data, outfile)

with open('corpORAORA.txt') as json_file:
    data = json.load(json_file)
    print(data[:10])
